## PALM Dash App

v2.0 (2021-3-3)  
James Harvey 

### Purpose
In this notebook we'll be creating a Dash App. The Dash app will be an interactive tool to play around with PALM Library that was developed by Suhas V. 

See PALM repo here: https://github.com/aifimmunology/PALM

### Contents

1. Load essential Libraries 
2. Run App 


### NOTES: 
- still need to do some setting up with packages: <br> 
    <b> NOTE: PALM has a few dependencies I need to install beforehand </b> 
    - sudo apt install cmake 
    - install.packages('nloptr') # reminder need to first install this via ubuntu: "sudo apt-get install libnlopt-dev"  
    - install.packages('lme4') 
    - install.packages('ggforce') 
    - install.packages('factoextra')
    - install.packages("/home/jupyter/PALM_Dash/PALM/PALM_0.1.0.tar.gz", repos = NULL, type ="source") # after cloning from repo 
- could potentially utilize long_callbacks when users want to run a particular graph with different parameters 
- TODO: get these in an image, or find some other way to do this set up under the hood 

___ 
new set up: 
<br> 
- sudo apt install libgeos-dev
- sudo apt install cmake 
- sudo apt-get install libnlopt-dev

R console: 
- library("devtools")
- install_github("aifimmunology/PALMO")
- library("PALMO")




___
### Essential Library for Dash App 

In [1]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()


___
<b> NOTE: additional step may be needed for public HISE: 
    - download repo or datasets locally 
___ 

TODO: 
- <b> ask users format of dataset.
       - convert files (.csv -> Rda)
    
- single-cell (H5 only) 

___ 

#### <b> Run App 

    This function allows you to perform analysis of longitudinal dataset. It requires longitudinal data matrix/data frame and annotation file.
    
The following are information about all the arguments you can pass in the below cell-block: 

- <b> metadata : str </b> -  A filepath to an annotation table. Table must consist column Sample (Participant sample name), PTID (Participant), Time (longitudinal time points)
- <b> data : str </b> - A filepath for an expression matrix or data frame. Rows represents gene/proteins column represents participant samples (same as annotation table Sample column). For single cell, Single cell RNA Seurat object, if datatype is single cell RNA and Single cell ATAC genescore matrix or data frame
- <b> datatype : str </b>-  Data input can be bulk or singlecell
- <b> feature_set : str </b>- Variance analysis carried out on the featureSet provided such as c("PTID", "Time", "Sex")
- <b> mean_threshold : float </b>- Average expression threshold to filter lowly expressed genes Default is 0.1 (log2 scale)
- <b> cv_threshold : float </b>- Coefficient of variation threshold to select variable and stable genes Default is 10 for single cell RNA (100*SD/mean)
- <b> na_threshold : float </b>-  Number of NAs in data (numeric value or NULL). Default, 40% * number of columns.
- <b> do_outlier : bool </b>- Whether to perform outlier analysis (TRUE or FALSE). Default FALSE
- <b> z_cutoff : float </b>- |Z| cutoff threshold to find potential outliers (Eg. z_cutoff= 2, equals to Mean/SD 2)
- <b> output_dir : str </b>-  User-defined output directory

    

    


In [3]:
%run app.py --metadata /home/jupyter/PALM_DASHv2/PALM/data/data_Metadata.Rda \
    --data /home/jupyter/PALM_DASHv2/PALM/data/Olink_NPX_log2_Protein.Rda \
    --datatype bulk \
    --do_outlier True \
    --z_cutoff 2 \
    --mean_threshold 1 \
    --cv_threshold 5 \
    --na_threshold 0.4 \
    --feature_set PTID Time \
    --output_dir /home/jupyter \
    --method spearman

ModuleNotFoundError: No module named 'diskcache'

_____
